<img style="float: center;" src="images/CI_horizontal.png" width="600">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Ghani, Rayid, Frauke Kreuter, Julia Lane, Brian Kim, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Avishek Kumar, Jonathan Morgan, Ekaterina Levitskaya.

# Dataset Exploration
----------

## Introduction

In an ideal world, we will have all of the data we want with all of the desirable properties (no missing values, no errors, standard formats, and so on). We'd also have perfect data documentation, with summary statistics and approproiate aggregate measures of everything we'd want to investigate. However, that is hardly ever true - and we have to work with using our datasets to answer questions of interest as intelligently as possible. 

In this notebook, we will discover the datasets we have on the ADRF, and we will use our datasets to answer some questions of interest. 

### Learning Objectives

This notebook will give you the opportunity to spend some hands-on time with the data. Throughout the notebook, we will demonstrate various techniques of how to use SQL and Python to explore the various datasets that we have and better understand what we are working with. This will form the basis of all the other types of analyses we do in this class and is a crucial first step for any data analysis workflow. As you work through the notebook, we will have checkpoints for you try out your own code, but you can also think about how you might apply any of the techniques and code presented with other datasets as well. What we are showing is just a portion of what you might be interested in investigating, so don't feel restricted by the questions we've decided to try to answer.

**Datasets We Will Explore In This Notebook**
- **Survey of Earned Doctorates (SED)**: individual level data (educational history, demographic, and postgraduation plans) of individuals receiving research doctoral degrees from U.S. academic institutions.
- **Survey of Doctorate Recipients (SDR)**: individual level data (demographic, education, and career history information from individuals with a U.S. research doctoral degree in a science, engineering, or health (SEH) field).
- **UMETRICS**: institution/individual level administrative universities' data.
- **HERD**: institution level data on R&D funding.

You will have an opportunity to explore the different datasets in the ADRF, and this notebook will take you around the different ways you can analyze your data. This involves looking at basic metrics in the larger dataset, taking a random sample, creating derived variables, making sense of the missing values, and so on. 

This will be done using both SQL and `pandas` in Python. The `sqlalchemy` Python package will give you the opportunity to interact with the database using SQL to pull data into Python. Some additional manipulations will be handled by Pandas in Python (by converting your datasets into dataframes).

**This notebook will provide an introduction and examples for:**

- How to create new tables from the larger tables in database (sometimes called the "analytical frame")
- How to explore different variables of interest
- How to explore aggregate metrics
- How to join tables

### Methods

We will be using the `sqlalchemy` Python package to access tables in our class database server - PostgreSQL. 

To read the results of our queries, we will be using the `pandas` Python package, which has the ability to read tabular data from SQL queries into a pandas DataFrame object. Within `pandas`, we will use various commands:

- subsetting data
- `groupby`
- `merge`

Within SQL, we will use various queries to:

- select data subsets
- sum over groups
- create new tables
- count distinct values of desired variables
- order data by chosen variables

## Python Setup

In Python, we `import` packages. The `import` command allows us to use libraries created by others in our own work by "importing" them. You can think of importing a library as opening up a toolbox and pulling out a specific tool. Among the most famous Python packages:
- **numpy** is short for "numerical Python". `numpy` is a lynchpin in Python's scientific computing stack. Its strengths include a powerful *N*-dimensional array object, and a large suite of functions for doing numerical computing. 
- **pandas** is a library in Python for data analysis that uses the DataFrame object (modeled after R DataFrames, for those familiar with that language) which is similiar to a spreedsheet but allows you to do your analysis programaticaly rather than the point-and-click of Excel. It is a lynchpin of the PyData stack and is built on top of `numpy`.  
- **sqlalchemy** is a Python library for interfacing with a PostGreSQL database. 

As we are working with the survey data, we will also use the **statsmodels** package in order to be able to apply survey weights in our calculations.

In [ ]:
# pandas-related imports
import pandas as pd

# Numpy 
import numpy as np

# database interaction imports
import sqlalchemy

# get weighted estimates
from statsmodels.stats.weightstats import DescrStatsW

__When in doubt, use shift + tab to read the documentation of a method by placing a cursor near the name of the method and pressing shift + tab.__

__The `help()` function also provides information on what you can do with a function.__

## Load the Data

We can execute SQL queries using Python to get the best of both worlds. For example, Python - and pandas in particular - make it much easier to calculate descriptive statistics and perform more complicated analyses with the data. Additionally, as we will see in the Data Visualization exercises, it is relatively easy to create data visualizations using Python. 

Pandas provides many ways to load data. It allows the user to read the data from a local csv or excel file, pull the data from a relational database, or read directly from a URL (when you have internet access). Since we are working with the PostgreSQL database `appliedda` in this course, we will demonstrate how to use pandas to read data from a relational database. For examples to read data from a CSV file, refert to the pandas documentation [Getting Data In/Out](pandas.pydata.org/pandas-docs/stable/10min.html#getting-data-in-out).

The function to run a SQL query and pull the data into a pandas dataframe (more to come) is `pd.read_sql()`. Just like doing a SQL query from pgAdmin, this function will ask for some information about the database, and what query you would like to run. Let's walk through the example below.

### Establish a Connection to the Database

The first parameter is the connection to the database. To create a connection we will use the SQLAlchemy package and tell it which database we want to connect to.

In [ ]:
# to create a connection to the database, 
# we need to pass the name of the database and host of the database

host = 'stuffed.adrf.info'
DB = 'appliedda'

connection_string = "postgresql://{}/{}".format(host, DB)
conn = sqlalchemy.create_engine(connection_string)

> Note we can parameterize Python `string` objects - using the built-in `.format()` function. We will use various formulations in the program notebooks (eg when building queries), some examples are:
1. Empty brackets (shown above) which simply inserts the variable in the string; when there is more than one set of brackets Python will insert variables in the order they are listed
2. Brackets with formatting can be used to make print statements more readable (eg `'text with formatted number with comma and 1-digit decimal {:,.1f}'.format(number_value)` will print `123,456.7` instead of `123456.7123401`)
3. Named brackets to use the same variables multiple times in a text block (we use this in more compicated queries eg when creating "labels" and "features" for Machine Learning models)

### Formulate Data Query

This part is similar to writing a SQL query in DBeaver. Depending on what data we are interested in, we can use different queries to pull different data. In this example, we will pull all the content from the SED data for doctoral students who graduated in 2015.

__Create a query as a `string` object in Python__

In [ ]:
query = '''
SELECT *
FROM ncses_2019.nsf_sed
WHERE phdfy = '2015'
LIMIT 10
'''

> The three quotation marks surrounding the query body is called multi-line string. It is quite handy for writing SQL queries because the new line character will be considered part of the string, instead of breaking the string

In [ ]:
# Now that we have defined a variable `query`, we can call it in the code
print(query)

> Note that the `LIMIT` provides one simple way to get a "sample" of data; however, using `LIMIT` does **not provide a _random_** sample. You may get different samples of data than others using just the `LIMIT` clause, but it is just based on what is fastest for the database to return.

### Pull Data from the Database

Now that we have the two parameters (database connection and query), we can pass them to the `pd.read_sql()` function, and obtain the data.

In [ ]:
# here we pass the query and the connection to the pd.read_sql() function and assign the variable `df`
# to the dataframe returned by the function
df = pd.read_sql(query, conn)

In [ ]:
df.head()

### What is in the Database?

**Schemas and Tables**

> As a reminder, in this class you have access to the following schemas: `ncses_2019` and `ada_ncses_2019`. You only have write privileges to the `ada_ncses_2019` schema.

In [ ]:
query = '''
SELECT tablename
FROM pg_tables
WHERE schemaname = 'ncses_2019'
'''
tables = pd.read_sql(query, conn)

In [ ]:
sorted(tables['tablename'])

Again, take some time to look at the documentation and understand what the different variables refer to.

<font color=red><h3> Checkpoint 1: Read in the table with SDR data</h3></font>

Similarly to the code above, read in and explore the table with SDR 2017 data (name: `nsf_sdr_2017`)

In [ ]:
query = '''
SELECT *
FROM ncses_2019.nsf_sdr_2017
'''
sdr_2017 = pd.read_sql(query,conn)

sdr_2017.head()

## Summary Statistics

In this section, we will start looking at aggregate statistics on the data. The goal of this exercise is to get a better understanding of the data we working with. As you work through this section, try to ask yourself some questions: Are the data generally clean? What are possible sources of error? What are the types of objects and variables that you are working with?

> Note: __Large tables__ can take a long time to process on shared databases, so we will demonstrate using SQL and Python with consideration for how much data we are reading back into Python

To answer these broader research questions, let's start by looking at simple aggregate statistics in each of our data sources.

### Data Exploration #1: **Survey of Earned Doctorates (SED)**

**Motivating Question:** What is a primary source of funding for doctorate students?

In order to avoid pulling a large amount of information that we don't need to answer the question, let's only
pull in the data with the unique identifier of a person (`drf_id`) and their primary source of support (in the SED data this variable is called `srceprim`, primary source of support).

In [ ]:
# Create the query and select only two variables: unique identifier (drfid) and primary source of support (srceprim)

query = '''
SELECT drf_id, srceprim
FROM ncses_2019.nsf_sed
WHERE phdfy = '2015'
'''

In [ ]:
# Read it into a pandas dataframe

sed_ncses_2015 = pd.read_sql(query,conn)

In [ ]:
# View the first rows of the table

sed_ncses_2015.head()

Let's check what are the unique values in our primary support variable. We can use `SELECT DISTINCT` in SQl.

In [ ]:
query = '''
SELECT DISTINCT(srceprim)
FROM ncses_2019.nsf_sed
WHERE phdfy = '2015'
'''
pd.read_sql(query,conn)

Use the `COUNT`, `GROUP BY` and `ORDER BY` functions in SQL to aggregate the number of graduates in each category and sort them in a descending order.

In [ ]:
# Count the number of graduates (their unique identifiers), group by a primary source of support variable, and sort 
# the counts in a descending order

query = '''
SELECT COUNT(drf_id), srceprim
FROM ncses_2019.nsf_sed
WHERE phdfy = '2015'
GROUP BY srceprim
ORDER BY COUNT(drf_id) DESC
'''

In [ ]:
primary_support = pd.read_sql(query,conn)

In [ ]:
primary_support  # call the name of the dataframe to view the results

It would be useful to see in the same table what those categories stand for - here we looked up the categories and created a separate dataframe, with which we will merge our dataframe above on the column with category letters.

In [ ]:
# Create a new dataframe with description of primary support categories

primary_source = pd.DataFrame()
primary_source['srceprim'] = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N']
primary_source['description'] = ['Fellowship, scholarship', 'Dissertation grant', 'Teaching assistantship', 'Research assistantship',
                                'Other assistantship','Traineeship','Internship, clinical residency', 'Loans (from any source)',
                                'Personal savings', 'Personal earnings during graduate school (other than sources listed above)',
                                'Spouse\'s, partner\'s, or family\'s earnings or savings', 'Employer reimbursement/assistance',
                                'Foreign (non-U.S.) support', 'Other - specify']

We use the `pandas` method `.merge()` to join two tables, specifying which column we want to combine the two DataFrame objects on.

In [ ]:
primary_source_merged = primary_source.merge(primary_support,on='srceprim')

In [ ]:
primary_source_merged

Let's also sort the values, with the largest value first - for that, we use `.sort_values()` function and specify a parameter `ascending=False`

In [ ]:
primary_source_sorted = primary_source_merged.sort_values('count',ascending=False)

In [ ]:
primary_source_sorted

What if we want to know a percentage of the cohort? To calculate that, we can get the sum of the column with number of graduates to find out the total number, using function `.sum()`.

In [ ]:
primary_source_sorted['count'].sum()

Now let's create a new column with this number.

In [ ]:
primary_source_sorted['total_cohort'] = [48419] * len(primary_source_sorted)

`pandas` allows for a very quick and easy calculation of columns in a dataframe - to find out the percentage, we will simply divide the column with the number of graduates by the total number of graduates and create a new column called `percentage` with those values 

In [ ]:
primary_source_sorted['percentage'] = primary_source_sorted['count'] / primary_source_sorted['total_cohort'] * 100

In [ ]:
primary_source_sorted

<font color=red><h3> Checkpoint 2: Find a secondary source of support</h3></font>

For the same cohort of 2015, repeat the code above and find a secondary source of support (variable: `srcesec`).

In [ ]:
# Select the variable of interest, count and group by a category for 2015 cohort and sort values

query = '''
SELECT COUNT(drf_id), srcesec
FROM ncses_2019.nsf_sed
WHERE phdfy = '2015'
GROUP BY srcesec
ORDER BY COUNT(drf_id) DESC
'''

secondary_support = pd.read_sql(query,conn)

secondary_support.head()

### Data Exploration #2: **UMETRICS**

**Motivating Question:** How important is federal funding for doctorate recipients?

UMETRICS data can help provide insight into the funding history of doctorate recipients. In the `semester` file, we can see a source which most frequently funds a given student in a given semester (variable `modal_funder`). 

To use the available information on funding history for the SED cohort 2015, we will need to join the tables. We'll do this using SQL code, and bring in the joined table into Python as a `DataFrame`.

In [ ]:
# First, we join the SED table using an SED-UMETRICS crosswalk, and then we join the resulting table with the IRIS semester table
# get get the name of the most frequently funding agency and the number of semesters 
query = '''
SELECT sed.drf_id, iris_semester.modal_funder, COUNT(iris_semester.semester) AS "number_semesters"
FROM ncses_2019.nsf_sed sed
JOIN ncses_2019.sed_umetrics_xwalk xwalk ON sed.drf_id = xwalk.drf_id
JOIN ncses_2019.iris_semester iris_semester ON iris_semester.emp_number = xwalk.emp_number
WHERE sed.phdfy = '2015'
GROUP BY sed.drf_id, iris_semester.modal_funder
'''
funding = pd.read_sql(query,conn)

In [ ]:
funding.head()

We can compare this information with the primary source of support variable which we explored above in the SED dataset.

In [ ]:
query = '''
SELECT drf_id, srceprim
FROM ncses_2019.nsf_sed
WHERE phdfy = '2015'
'''
source_support_sed = pd.read_sql(query,conn)

In [ ]:
source_support_sed.head()

In [ ]:
# Merge with the table defined above with categories' definitions
source_support = source_support_sed.merge(primary_source,on='srceprim')

In [ ]:
source_support.head()

Now we can merge with the IRIS data on funding defined above.

In [ ]:
funding_comparison = source_support.merge(funding,on='drf_id')

In [ ]:
funding_comparison.head()

We can subset by an individual:

In [ ]:
funding_comparison[funding_comparison['drf_id'] == 'XXX']

Thanks to a joined SED-UMETRICS dataset, we now have a more detailed picture of a person's funding history. DELETED DURING DISCLOSURE REVIEW.

<font color=red><h3> Checkpoint 3: Find number of semesters by non-federal source of funding.</h3></font>

IRIS `semester` file has a flag for non-federal sources of funding called `any_non_federal`. Find the number of semesters where `any_non_federal` source of funding is True (equals 1).

In [ ]:
query = '''
SELECT *
FROM ncses_2019.iris_semester
WHERE any_non_federal = 1
'''

non_federal_funding = pd.read_sql(query,conn)

In [ ]:
non_federal_funding.head()

### Data Exploration #3: **Survey of Doctorate Recipients (SDR)**

**Motivating Question:** What is the distribution of earnings by gender and by race/ethnicity?

As in the SDR data we are working with sub-samples of the SED population, we will need to use survey weights in our calculations.

Let's find the distribution of earnings for the SED cohort 2015. In the SDR data, we will use the variable `sdryr` (the year of first award of a U.S. PhD degree) to subset by year 2015, and we will also use `salary`, `gender`, and `wtsurvy` variables.

In [ ]:
# Let's get the relevant variables from the SDR data to find the female earnings among the 2015 cohort

query = '''
SELECT salary, wtsurvy
FROM ncses_2019.nsf_sdr_2017
WHERE sdryr = '2015' 
AND gender = 'F'
'''
female_earnings = pd.read_sql(query,conn)

In [ ]:
female_earnings.head()

We will apply the `DescrStatsW` function to calculate the weighted earnings distribution.

In [ ]:
weighted_female_earnings = DescrStatsW(female_earnings.salary, weights=female_earnings.wtsurvy)

To find the percentiles, we will use a built-in `pandas` function `.quantile()`.

In [ ]:
weighted_female_earnings.quantile([.1, .25, .5, .75, .9])

<font color=red><h3> Checkpoint 4: Find the distribution of earnings by race/ethnicity</h3></font>

Using the `DescrStatsW` function above, find the distribution of earnings for the Hispanic population for the cohort 2015 (variable `racethm = '4'`).

In [ ]:
query = '''
SELECT salary, wtsurvy
FROM ncses_2019.nsf_sdr_2017
WHERE sdryr = '2015' 
AND racethm = '4'
'''
earnings_hispanic = pd.read_sql(query,conn)

earnings_hispanic.head()

In [ ]:
# Find the weighted estimates
weighted_earnings_hispanic = DescrStatsW(earnings_hispanic.salary, weights=earnings_hispanic.wtsurvy)

In [ ]:
# Find the percentiles
weighted_earnings_hispanic.quantile([.1, .25, .5, .75, .9])

### Data Exploration #4: **Higher Education Research and Development Survey (HERD)**

**Motivating Question:** What are the institutional characteristics of the various schools from which graduate students receive their PhDs?

To answer this question, let's explore the HERD data.

In [ ]:
query = '''
SELECT *
FROM ncses_2019.nsf_herd
'''

In [ ]:
herd = pd.read_sql(query,conn)

In [ ]:
herd.head()

The HERD data has flags for whether the university has a medical school. The values are stored as `boolean`:
    `True` or `False` (for whether a university has a medical school).

Count the number of universities with medical school:

In [ ]:
query = '''
SELECT COUNT(std_inst_name)
FROM ncses_2019.nsf_herd
WHERE med_sch_flag = 'T'
'''
pd.read_sql(query,conn)

We can do the same in `pandas` by subsetting the dataframe:

In [ ]:
med_school_flag = herd[herd['med_sch_flag'] == 'T']

And counting the total number of rows of universities with a medical school by calling `len` (length of a dataframe):

In [ ]:
len(herd[herd['med_sch_flag'] == 'T'])

<font color=red><h3> Checkpoint 4: Explore the HERD data</h3></font>

1. Find how many universities are flagged as historically black colleges and universities (variable: `hbcu_flag`).

In [ ]:
query = '''
SELECT COUNT(std_inst_name)
FROM ncses_2019.nsf_herd
WHERE hbcu_flag = 'T'
'''
pd.read_sql(query,conn)

2. Order by total R&D funding in descending order.

In [ ]:
query = '''
SELECT COUNT(std_inst_name)
FROM ncses_2019.nsf_herd
GROUP BY srcesec
ORDER BY COUNT(drf_id) DESC
'''
pd.read_sql(query,conn)

> Reminder: you can refer to the documentation for more information on each dataset